In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests

api_key = "57CA4651BA0DEA92783B179593FBE41A"
app_id = 271590  # Ejemplo: GTA V
url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
response = requests.get(url)
data = response.json()
print(data[str(app_id)]["data"]["name"])  # Imprime el nombre del juego

Grand Theft Auto V Legacy


In [4]:
import requests
url = "https://steamspy.com/api.php?request=appdetails&appid=271590"
response = requests.get(url)
print(response.json())  # Datos como número de jugadores, tiempo de juego, etc.

{'appid': 271590, 'name': 'Grand Theft Auto V Legacy', 'developer': 'Rockstar North', 'publisher': 'Rockstar Games', 'score_rank': '', 'positive': 1734968, 'negative': 250463, 'userscore': 0, 'owners': '50,000,000 .. 100,000,000', 'average_forever': 13601, 'average_2weeks': 530, 'median_forever': 5884, 'median_2weeks': 169, 'price': '0', 'initialprice': '0', 'discount': '0', 'ccu': 92682, 'languages': 'English, French, Italian, German, Spanish - Spain, Korean, Polish, Portuguese - Brazil, Russian, Traditional Chinese, Japanese, Simplified Chinese, Spanish - Latin America', 'genre': 'Action, Adventure', 'tags': {'Open World': 32669, 'Action': 23556, 'Multiplayer': 21765, 'Crime': 19440, 'Automobile Sim': 18783, 'Third Person': 18676, 'First-Person': 18287, 'Mature': 15311, 'Shooter': 14967, 'Adventure': 13655, 'Singleplayer': 12524, 'Third-Person Shooter': 11999, 'Racing': 11588, 'Co-op': 10283, 'Sandbox': 10012, 'Atmospheric': 9943, 'Funny': 9410, 'Great Soundtrack': 8744, 'Comedy': 86

In [11]:
import requests
from bs4 import BeautifulSoup
import time
import json

# Lista de AppIDs (puedes agregar más)
app_ids = [271590, 1085660, 1172470]  # GTA V, Destiny 2, Apex Legends
results = []

# Idiomas permitidos para las reseñas de Steam
ALLOWED_REVIEW_LANGUAGES = ["english", "spanish"]

# --- Define el mapeo de AppID a nombre fuera del bucle ---
app_id_to_name_map = {
    271590: "grand-theft-auto-v",
    1085660: "destiny-2",
    1172470: "apex-legends"
}
# --------------------------------------------------------

for app_id in app_ids:
    print(f"Procesando AppID: {app_id}")

    # Obtener precio actual (Steam API)
    price_url = f"https://store.steampowered.com/api/appdetails?appids={app_id}&currency=mx&filters=price_overview,is_free" # Añadir is_free filter
    price_data = {}
    try:
        price_response = requests.get(price_url, headers={"User-Agent": "Mozilla/5.0"})
        price_response.raise_for_status()
        price_data = price_response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener datos de precio para {app_id}: {e}")
    except json.JSONDecodeError:
        print(f"Error al decodificar JSON para datos de precio {app_id}")


    # Obtener reseñas de usuarios (Steam API)
    review_url = f"https://store.steampowered.com/appreviews/{app_id}?json=1&language=all&num_per_page=10" # Aumentar a 10 para tener más variedad para filtrar
    review_data = {}
    try:
        review_response = requests.get(review_url, headers={"User-Agent": "Mozilla/5.0"})
        review_response.raise_for_status()
        review_data = review_response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener reseñas de usuario para {app_id}: {e}")
    except json.JSONDecodeError:
         print(f"Error al decodificar JSON para reseñas de usuario {app_id}")

    # Obtener reseñas profesionales (Metacritic, parseo ligero)
    # --- Usar el mapeo definido fuera del bucle ---
    game_name = app_id_to_name_map.get(app_id, "unknown")
    # --------------------------------------------
    metacritic_url = f"https://www.metacritic.com/game/{game_name}/"
    metascore = "N/A"
    user_score = "N/A"
    if game_name != "unknown":
        try:
            metacritic_response = requests.get(metacritic_url, headers={"User-Agent": "Mozilla/5.0"})
            metacritic_response.raise_for_status()
            if metacritic_response.status_code == 200:
                soup = BeautifulSoup(metacritic_response.text, "html.parser")

                # Metascore - Buscar el span directamente
                metascore_elem = soup.find("span", class_="c-score-box metascore_w")
                if metascore_elem:
                     metascore = metascore_elem.text.strip()

                # User Score - Buscar el contenedor primero, luego el span dentro
                user_score_container = soup.find("div", class_="c-score-box c-score-box--user")
                if user_score_container: # Comprobación añadida aquí
                    # Intentar encontrar la puntuación numérica
                    user_score_elem = user_score_container.find("span", class_="u-fontWeight-bold")
                    if user_score_elem:
                         user_score = user_score_elem.text.strip()
                    else:
                        # Si no es numérica, intentar encontrar el caso "tbd"
                        user_score_elem_tbd = user_score_container.find("span", class_="c-score-box__text--tbd")
                        if user_score_elem_tbd:
                             user_score = user_score_elem_tbd.text.strip()


        except requests.exceptions.RequestException as e:
            print(f"Error al obtener datos de Metacritic para {game_name}: {e}")
        except Exception as e:
             print(f"Error al parsear Metacritic para {game_name}: {e}")


    # Procesar datos
    result = {
        "app_id": app_id,
        "price": "N/A",
        "discount": 0,
        "user_reviews": [], # Esta lista se llenará solo con los idiomas permitidos
        "metascore": metascore,
        "metacritic_user_score": user_score
    }

    # ----- Sección de precio corregida para manejar juegos gratuitos -----
    app_data = price_data.get(str(app_id))
    if app_data and isinstance(app_data, dict) and app_data.get("success"):
        data_section = app_data.get("data")
        if data_section and isinstance(data_section, dict):
            # Verificar si el juego es gratuito
            if data_section.get("is_free"):
                result["price"] = "Free"
                result["discount"] = 0 # O podrías poner "N/A" o None

            # Si no es gratuito, buscar la información de precio
            else:
                price_info = data_section.get("price_overview")
                if price_info and isinstance(price_info, dict):
                    result["price"] = price_info.get("final_formatted", "N/A")
                    result["discount"] = price_info.get("discount_percent", 0)
    # --------------------------------------------------------------------


    # ----- Sección de reseñas de usuarios corregida para filtrar por idioma -----
    if review_data.get("success") == 1:
        reviews_list = review_data.get("reviews", [])
        if isinstance(reviews_list, list):
            filtered_reviews = [] # Creamos una lista temporal para las reseñas filtradas
            for r in reviews_list:
                if isinstance(r, dict):
                    review_language = r.get("language", "unknown") # Obtener el idioma de la reseña

                    # --- FILTRAR POR IDIOMA ---
                    if review_language in ALLOWED_REVIEW_LANGUAGES:
                        author_info = r.get("author", {})
                        review_text = r.get("review", "")
                        voted_up = r.get("voted_up", False)

                        filtered_reviews.append({
                            "author": author_info.get("steamid", "N/A"),
                            "text": review_text[:200] + "..." if len(review_text) > 200 else review_text,
                            "rating": "Positivo" if voted_up else "Negativo",
                            "language": review_language # Opcional: añadir el idioma al resultado
                        })
            result["user_reviews"] = filtered_reviews # Asignamos la lista filtrada
    # -----------------------------------------------------------------------------

    results.append(result)
    time.sleep(2)

# Imprimir resultados
for result in results:
    # --- Usar el mapeo definido fuera del bucle para obtener el nombre ---
    display_game_name = app_id_to_name_map.get(result['app_id'], 'Unknown Game')
    print(f"\n--- AppID: {result['app_id']} ({display_game_name}) ---")
    # -----------------------------------------------------------------
    print(f"Precio: {result['price']}, Descuento: {result['discount']}%")
    print(f"Metascore: {result['metascore']}, User Score (Metacritic): {result['metacritic_user_score']}")
    print(f"Reseñas de usuarios (Steam) - ({'/'.join(ALLOWED_REVIEW_LANGUAGES)}):")
    if result["user_reviews"]:
        for review in result["user_reviews"]:
             print(f"- {review['rating']}: {review['text']} (Usuario: {review['author']})")
    else:
        print("  No hay reseñas disponibles en los idiomas solicitados.")

Procesando AppID: 271590
Procesando AppID: 1085660
Procesando AppID: 1172470

--- AppID: 271590 (grand-theft-auto-v) ---
Precio: N/A, Descuento: 0%
Metascore: N/A, User Score (Metacritic): N/A
Reseñas de usuarios (Steam) - (english/spanish):
- Negativo: This WAS a fun game, but then rockstar decided to "enhance" their game with kernel level spyware because they don't understand what server side anti cheat is. Then they also force this spyware on peop... (Usuario: 76561197993138592)
- Positivo: [h1] Grand Theft Auto V Legacy [/h1]

[h3] ---{ Graphics }--- [/h3]
☐ You forget what reality is
☑ Beautiful
☐ Good
☐ Decent
☐ Bad
☐ Don‘t look too long at it
☐ MS-DOS

[h3] ---{ Gameplay }--- [/h3]
☑... (Usuario: 76561198986968728)
- Negativo: If corruption, inbreeding and greed were the only qualities needed for one to be elected as a politician, Rockstar could form its own government, inhabited by the descendants of the apes, which still ... (Usuario: 76561198313664982)
- Negativo: Most overra